# Directed Search with Piecewise Pi

James Yu, 6 February 2021

[This PDF](https://montoya.econ.ubc.ca/Econ306/directed_search.pdf) by Dr. Michael Peters describes a problem of two firms and two workers where workers are looking to maximize wages and firms are setting wages to maximize profits. In this brief notebook, we expand on the problem where each worker is given a "high" or "low" type, and work out the calculations for the profit-maximizing wage choices in this case.

In [1]:
import numpy as np
import scipy.linalg as la
from sympy import *

We use Python's [NumPy](https://numpy.org/) and [SciPy](https://www.scipy.org/) libraries to make working with numerical data faster. We will need numerical data in order to solve the problem.

We also use the [SymPy](https://www.sympy.org/en/index.html) Python library, which functions similar to SageMath if you are familiar with that.

First, we establish the mixed-strategy Nash equilibrium $\pi$ value given in the original reading for the typed worker problem. This is obtained in the reading by setting the expected payoffs for each worker equal to each other. The problem is symmetric, so we only need to do this once.

Next, we do a similar computation for constraints on $\pi$ that was done for the basic typeless problem. Recall we had conditions where $\pi$ was greater than one or $\pi$ was negative, which each manifested as components of a final piecewise $\pi^*$. Instead of using these piecewise conditions directly, we can use the fact that we are essentially constraining $\pi$ to the interval $[0, 1]$, and can thus use `max()` and `min()` functions to do this work for us in Python.

Finally, given our upgraded $\pi$, we can plug it into the profit functions for firm 1 and 2. This leads to the following code:

In [2]:
def profit_1(w1, w2, λ, y1):
    π = min(1, max(0, (λ*(2*w1+w2)-(2*w1-w2))/((λ-1)*(w1+w2))))
    return (y1-w1)*(1-(1-π)**2)

def profit_2(w2, w1, λ, y2):
    π = min(1, max(0, (λ*(2*w1+w2)-(2*w1-w2))/((λ-1)*(w1+w2))))
    return (y2-w2)*(1-π**2)

λ, w1, w2 = symbols("λ, w1, w2")
(λ*(2*w1+w2)-(2*w1-w2))/((λ-1)*(w1+w2))

(-2*w1 + w2 + λ*(2*w1 + w2))/((w1 + w2)*(λ - 1))

Notice how we took the minimum of 1 and a function that gave the maximum of 0 and $\pi$. The maximum of 0 and $\pi$ ensures $\pi$ is never negative, and the minimum of 1 and this result ensures $\pi$ never exceeds 1.

Our profit functions are the same as in the readings, derived by subtracting wages from outputs and multiplying this by the probability of hiring a worker for the particular firm.

Next, we apply a few NumPy-related tricks. First, we vectorize each function to make them operate faster on matrices. We then establish a few sequences of numbers so we can try evaluating our profit functions for each parameter.

In [3]:
profit_1 = np.vectorize(profit_1)
profit_2 = np.vectorize(profit_2)

w1s = np.linspace(0.00000001, 1, 100)
w2s = np.linspace(0.00000001, 1, 100)
λs = np.linspace(0.00000001, 0.9999, 100)
ys = np.linspace(0, 2, 5)

Each linspace declares a start-point and end-point, along with a third parameter for number of entries. Given these ranges for our parameters, we can evaluate the profit function for combinations of them.

(Kudos to [this StackOverflow post](https://stackoverflow.com/questions/22774726/numpy-evaluate-function-on-a-grid-of-points) which had an implementation for evaluating functions over an entire matrix at once.)

In [4]:
result = profit_1(w1s[:,None,None,None], w2s[None,:,None,None], λs[None,None,:,None], ys[None,None,None,:])

This gives us a nice big matrix containing profits for firm 1 given various combinations of $w_1$, $w_2$, $y_1$ and $\lambda$. To find a best-response, we find the individual $w_1$ values which maximize profits for each $w_2$. That is, each $w_2$ present in the matrix corresponds to a list of $w_1$ values, so we pick the $w_1$ which maximizes profits to declare a best-response.

We don't need the actual profit value; rather, we need the index of the particular $w_1$ per $w_2$, so we use an argmax function.

In [5]:
firm1best = np.argmax(result, axis=0)

Now we repeat this for the other firm.

In [6]:
result2 = profit_2(w2s[:,None,None,None], w1s[None,:,None,None], λs[None,None,:,None], ys[None,None,None,:])
firm2best = np.argmax(result2, axis=0)

Given we have best-responses for firm 1, and also best-responses for firm 2, we can find a Nash equilibrium by finding the $w_1$-$w_2$ pairs which coincide in both best-response matrices. That is, if the best-response for firm 1 is to play $x$ when firm 2 plays $y$, and the best-response for firm 2 is to play $y$ when firm 1 plays $x$, this is a Nash equilibrium as no firm will be better off by deviating unilaterally.

To do this, we iterate over all combinations of $\lambda$, $y_1$ and $y_2$ possible to see what the best-responses are in each scenario. Our final profit function must depend on these three variables since these are the three original givens, and so we have to check all of them.

In [7]:
res = []
for λ in range(len(λs)):
    for y1 in range(len(ys)):
        for y2 in range(len(ys)):
            f1response = firm1best[:,λ,y1]
            f2response = firm2best[:,λ,y2]
            for i in f1response:
                if f1response[f2response[i]] == i:
                    res.append((w1s[i], w2s[f2response[i]], λs[λ], ys[y1], ys[y2]))
                        
print(len(res))

60730


Note the quadruple for loop, which is generally frowned upon universally. It computes fast enough for our purposes, however.

Some of the points may be duplicated because of symmetry and possible rounding so let's try to run `set()` to remove duplicates. This function turns a list into a set of unique elements.

In [8]:
data = list(set(res))
print(len(data))

3025


That's much smaller, right? 

We now have a vector containing a large list of Nash equilibria for various $y_1$, $y_2$, $\lambda$ pairs. This doesn't do well if we have a $y_1$ or $y_2$ out of the range of $[0, 2]$, so let's try to construct an actual best-response function using our data.

We use a technique which can probably indisputably be called multivariate polynomial regression. As we have three independent variables and a univariate output, we need some sort of three-dimensional interpolation method to fit a function to our data. The existing literature on this provides trilinear and tricubic interpolation, which either doesn't provide a high enough degree of polynomial or isn't outlined in enough detail to implement quickly. Instead, we construct an entirely different method which is far more questionable but nonetheless seems to work.

For reference, we are deriving the model based on the Polynomial Interpolation and Least Squares Linear Regression sections of [this webpage](https://www.math.ubc.ca/~pwalls/math-python/linear-algebra/applications/#polynomial-interpolation) in Dr. Patrick Walls' *Mathematical Python*. We take this model and expand it to become multivariate by adding extra entries in our data matrix to account for the extra variables. Before continuing, give the page a read.

Whereas in the existing model each row of the matrix corresponds to an $n$-th degree univariate polynomial, giving $n+1$ entries in the row, here each row corresponds to an $n$-th degree trivariate polynomial with $(n+1)^3$ entries in each row. Each entry is now of the form $c_{ijk} * \lambda^i * y_1^j * y_2^k$ for $i, j, k$ in $[0, n]$ and some coefficient $c_{ijk}$. This gives us all possible variable combinations in such a trivariate polynomial.

Given one of these rows for every data point, we solve a matrix system for all the coefficients $c$ which, when applied to all of our data points, retrieves the best-response $w_1$ or $w_2$ we need. That is, on top of the original interpolation model, $y_m$ contains our best response wage $w_{1_m}$ for the $\lambda$, $y_1$, $y_2$ combination represented in the $m$-th row of the matrix.

The original interpolation model demands a square matrix in order to have a definitive solution for the matrix system. When not possible, the alternative of least squares regression can be used to determine an approximation. As we have a few thousand data points to fit, and we would like to not have a 3000-th degree polynomial as our best response function, we'll use this approximation and see how it goes.

First, we'll try for a quadratic polynomial. Our goal is to both get a solution and to make it as precise as possible.

In [9]:
def generate_matrix(n, data, wage):
    n = n + 1
    A = np.zeros((len(data), n**3))
    y = [entry[wage-1] for entry in data]

    index = 0
    for entry in data:
        λ = entry[2]
        y1 = entry[3]
        y2 = entry[4]
        vec = []
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    vec.append(λ**k * y1**j * y2**i)

        A[index,:] = vec[:]
        index += 1
    
    return A, y

A, y = generate_matrix(2, data, 1)

The above code generates the $A$ matrix containing our data points. We now run least squares regression to see if we can come up with coefficients.

In [10]:
c = la.lstsq(A,y)
print(c[0])

[-6.73576649e-04  3.90242874e-02 -4.80522114e-02  6.20567477e-02
  8.72881155e-02  6.56752631e-02 -2.83074280e-02  1.45469432e-02
 -6.50605531e-02  5.23908935e-02 -1.87929362e-01  1.28428427e-01
  7.23425519e-01  9.44259191e-01  6.46494828e-01 -1.34068690e-01
  7.87213798e-03 -6.50367244e-01 -2.31250666e-02  5.00174205e-02
 -2.15491367e-02 -2.04235070e-01 -1.22315722e-01 -5.78023781e-01
  4.93069738e-02 -1.93359538e-01  4.53023728e-01]


A bunch of fancy numbers. We can recompile this into a symbolic polynomial.

In [11]:
def recompile(n, coeff):
    n = n + 1
    eq = 0
    index = 0
    λ, y1, y2 = symbols("λ, y1, y2")
    for i in range(n):
        for j in range(n):
            for k in range(n):
                eq += coeff[index] * λ**k * y1**j * y2**i
                index += 1

    return eq

polynomial = recompile(2, c[0])
polynomial

0.453023728163665*y1**2*y2**2*λ**2 - 0.193359538246627*y1**2*y2**2*λ + 0.0493069737513751*y1**2*y2**2 - 0.650367243505001*y1**2*y2*λ**2 + 0.00787213798318682*y1**2*y2*λ - 0.134068689958266*y1**2*y2 - 0.0650605531238954*y1**2*λ**2 + 0.0145469431771775*y1**2*λ - 0.0283074279810142*y1**2 - 0.57802378124725*y1*y2**2*λ**2 - 0.122315721632701*y1*y2**2*λ - 0.204235070158013*y1*y2**2 + 0.646494828370956*y1*y2*λ**2 + 0.944259191007472*y1*y2*λ + 0.723425519018*y1*y2 + 0.0656752631351035*y1*λ**2 + 0.0872881154700015*y1*λ + 0.0620567477467852*y1 - 0.0215491366745882*y2**2*λ**2 + 0.0500174205283231*y2**2*λ - 0.0231250665793527*y2**2 + 0.128428427139525*y2*λ**2 - 0.18792936231927*y2*λ + 0.0523908935001176*y2 - 0.0480522114251742*λ**2 + 0.0390242874332167*λ - 0.000673576649039597

The decimals are long but this gives a symbolic equation! But before we celebrate, let's check the error level. We can compute both absolute maximum error and average error by comparing our original data to the data generated by this polynomial.


In [12]:
absolute_error = 0
avg_error = 0


def update_error(k, wage):
    global absolute_error, avg_error
    λ, y1, y2 = symbols("λ, y1, y2")
    for dat in range(k*len(data)//4, (k+1)*len(data)//4):
        e = data[dat]
        error = abs(polynomial.subs([[λ, e[2]], [y1, e[3]], [y2, e[4]]]) - e[wage-1])
        if error > absolute_error:
            absolute_error = error

        avg_error += error

In [13]:
update_error(0, 1)

In [14]:
update_error(1, 1)

In [15]:
update_error(2, 1)

In [16]:
update_error(3, 1)

Unfortunately we have to split that into four parts because otherwise GitHub would never finish rendering in time. We can now print our error data.

In [17]:
print(avg_error / len(data))
print(absolute_error)

0.0606174624387796
0.280149851727390


That's okay, but not great. This says on average, our best-response wage may be up to 0.061 off from the actual, and at worst may be 0.28 off from the actual.

We could try making a more complicated polynomial to see if it would be any better. Let's try a quartic one.

In [18]:
A, y = generate_matrix(4, data, 1)
c = la.lstsq(A,y)
polynomial = recompile(4, c[0])
polynomial

-30.5568317464197*y1**4*y2**4*λ**4 + 54.654435357051*y1**4*y2**4*λ**3 - 30.5428633789734*y1**4*y2**4*λ**2 + 5.64106905300703*y1**4*y2**4*λ - 0.148859543272948*y1**4*y2**4 + 129.12231765545*y1**4*y2**3*λ**4 - 225.900617493666*y1**4*y2**3*λ**3 + 122.618340789852*y1**4*y2**3*λ**2 - 21.7391777855753*y1**4*y2**3*λ + 0.484099501500281*y1**4*y2**3 - 160.245740043797*y1**4*y2**2*λ**4 + 268.307038941293*y1**4*y2**2*λ**3 - 137.950182469961*y1**4*y2**2*λ**2 + 22.8422230231961*y1**4*y2**2*λ - 0.320432428599*y1**4*y2**2 + 49.7014218553912*y1**4*y2*λ**4 - 74.6773755885589*y1**4*y2*λ**3 + 34.3573685591562*y1**4*y2*λ**2 - 4.96958497036158*y1**4*y2*λ - 0.0894794004080017*y1**4*y2 - 7.1932308026712e-10*y1**4*λ**4 + 1.5791024876588e-9*y1**4*λ**3 - 1.2717324970879e-9*y1**4*λ**2 + 4.29509067201295e-10*y1**4*λ - 0.006734006717207*y1**4 + 108.272144693775*y1**3*y2**4*λ**4 - 190.295815513907*y1**3*y2**4*λ**3 + 104.311126938804*y1**3*y2**4*λ**2 - 18.9510967551032*y1**3*y2**4*λ + 0.408302841326196*y1**3*y2**4 -

In [19]:
absolute_error = 0
avg_error = 0
#update_error(0, 1)

In [20]:
#update_error(1, 1)

In [21]:
#update_error(2, 1)

In [22]:
#update_error(3, 1)

We didn't run `update_error()` explicitly because doing so would stop the notebook from loading. You can uncomment the above code if you'd like to try it yourself. For the quartic polynomial, we got an average error of about 0.005 and an absolute error of about 0.057. This is much better than what we had before, and if this were dollar-valued, on average we would essentially have no error on average since we denominate in cents, or 0.01-intervals. The maximum error is not great, but certainly better than before.

Personal testing revealed a 9th degree polynomial provided worse error than this 4th degree one, so higher isn't necessarily better. A 6th degree polynomial refuses to solve in this notebook entirely, though, so we won't explore it. You can try it on your own setup if you'd like.

Since we have a decent best-response function for firm 1's $w_1$, let's do it again for $w_2$.

In [23]:
A, y = generate_matrix(4, data, 2)
c2 = la.lstsq(A,y)
d = c2[0] 
for i in range(len(d)):
    if d[i] < 10**(-10):
        d[i] = 0
polynomial2 = recompile(4, c2[0])

In [24]:
polynomial2

15.4382699399085*y1**4*y2**4*λ**3 + 2.10410712835397*y1**4*y2**4*λ + 32.0519815400813*y1**4*y2**3*λ**4 + 37.782384525771*y1**4*y2**3*λ**2 + 0.12518047412843*y1**4*y2**3 + 65.3486989247366*y1**4*y2**2*λ**3 + 8.32561796753057*y1**4*y2**2*λ + 0.0575866522821714*y1**4*y2**2 + 6.91828695303802*y1**4*y2*λ**4 + 8.55770798688695*y1**4*y2*λ**2 + 8.98558921713644e-10*y1**4*λ**3 + 1.93779936541461e-10*y1**4*λ + 28.7576705836738*y1**3*y2**4*λ**4 + 34.0195637839138*y1**3*y2**4*λ**2 + 0.126500444756716*y1**3*y2**4 + 211.155363826836*y1**3*y2**3*λ**3 + 27.5936570089693*y1**3*y2**3*λ + 115.033899634319*y1**3*y2**2*λ**4 + 126.485700879125*y1**3*y2**2*λ**2 + 30.6702492017721*y1**3*y2*λ**3 + 5.1464407562741*y1**3*y2*λ + 1.44281396705288*y1**3*y2 + 1.59104607266869e-9*y1**3*λ**4 + 2.48679610237446e-9*y1**3*λ**2 + 56.3102085941945*y1**2*y2**4*λ**3 + 7.59896743091638*y1**2*y2**4*λ + 112.468921660882*y1**2*y2**3*λ**4 + 123.869802505736*y1**2*y2**3*λ**2 + 180.084297207485*y1**2*y2**2*λ**3 + 25.5875536963515*y

Some of the terms had incredibly small coefficients, so we tried setting them to zero when they were small. We can try doing the same to the other polynomial to see if it might simplify it a little.

In [25]:
d = c[0]
for i in range(len(d)):
    if d[i] < 10**(-10):
        d[i] = 0
polynomial = recompile(4, c[0])

In [26]:
polynomial

54.654435357051*y1**4*y2**4*λ**3 + 5.64106905300703*y1**4*y2**4*λ + 129.12231765545*y1**4*y2**3*λ**4 + 122.618340789852*y1**4*y2**3*λ**2 + 0.484099501500281*y1**4*y2**3 + 268.307038941293*y1**4*y2**2*λ**3 + 22.8422230231961*y1**4*y2**2*λ + 49.7014218553912*y1**4*y2*λ**4 + 34.3573685591562*y1**4*y2*λ**2 + 1.5791024876588e-9*y1**4*λ**3 + 4.29509067201295e-10*y1**4*λ + 108.272144693775*y1**3*y2**4*λ**4 + 104.311126938804*y1**3*y2**4*λ**2 + 0.408302841326196*y1**3*y2**4 + 775.321248598674*y1**3*y2**3*λ**3 + 71.6279613372878*y1**3*y2**3*λ + 550.236418650686*y1**3*y2**2*λ**4 + 447.023183967335*y1**3*y2**2*λ**2 + 0.225633108906552*y1**3*y2**2 + 229.415696370304*y1**3*y2*λ**3 + 13.3580144441111*y1**3*y2*λ + 0.838618550756612*y1**3*y2 + 2.60793164841289e-9*y1**3*λ**4 + 4.55642162888736e-9*y1**3*λ**2 + 0.0336700335086424*y1**3 + 192.954045327258*y1**2*y2**4*λ**3 + 18.8925111913297*y1**2*y2**4*λ + 460.879935076304*y1**2*y2**3*λ**4 + 403.162816703358*y1**2*y2**3*λ**2 + 0.0987096811993122*y1**2*y2*

Smaller, perhaps, but still large. This doesn't matter to too much of an extent. We can now do some interesting math.
First, let's use the best-response functions to simplify $\pi$.

In [27]:
w1, w2, λ = symbols("w1, w2, λ")
π = (λ*(2*w1+w2)-(2*w1-w2))/((λ-1)*(w1+w2))
π = π.subs(w1, polynomial).subs(w2, polynomial2)
simplify(π)

(-93.8706007741935*y1**4*y2**4*λ**3 - 9.17803097766009*y1**4*y2**4*λ - 226.19265377082*y1**4*y2**3*λ**4 - 207.454297053934*y1**4*y2**3*λ**2 - 0.843018528872131*y1**4*y2**3 - 471.265378957849*y1**4*y2**2*λ**3 - 37.3588280788617*y1**4*y2**2*λ + 0.0575866522821714*y1**4*y2**2 - 92.4845567577443*y1**4*y2*λ**4 - 60.1570291314254*y1**4*y2*λ**2 - 2.25964605360396e-9*y1**4*λ**3 - 6.65238197861129e-10*y1**4*λ - 187.786618803877*y1**3*y2**4*λ**4 - 174.602690093693*y1**3*y2**4*λ**2 - 0.690105237895676*y1**3*y2**4 - 1339.48713337051*y1**3*y2**3*λ**3 - 115.662265665606*y1**3*y2**3*λ - 985.438937667053*y1**3*y2**2*λ**4 - 767.560667055545*y1**3*y2**2*λ**2 - 0.451266217813104*y1**3*y2**2 - 428.161143538836*y1**3*y2*λ**3 - 21.5695881319482*y1**3*y2*λ - 0.234423134460347*y1**3*y2 - 3.62481722415708e-9*y1**3*λ**4 - 6.62604715540027e-9*y1**3*λ**2 - 0.0673400670172847*y1**3 - 329.597882060321*y1**2*y2**4*λ**3 - 30.186054951743*y1**2*y2**4*λ - 809.290948491726*y1**2*y2**3*λ**4 - 682.45583090098*y1**2*y2**3*

That's big. We can try making some profit functions now.

In [28]:
π = Min(1, Max(0, π))
y1, y2 = symbols("y1, y2")
w1 = polynomial
w2 = λ
profit_1 = (y1-w1)*(1-(1-π)**2)
profit_1

(1 - (1 - Min(1, Max(0, (-93.8706007741935*y1**4*y2**4*λ**3 - 9.17803097766009*y1**4*y2**4*λ - 226.19265377082*y1**4*y2**3*λ**4 - 207.454297053934*y1**4*y2**3*λ**2 - 0.843018528872131*y1**4*y2**3 - 471.265378957849*y1**4*y2**2*λ**3 - 37.3588280788617*y1**4*y2**2*λ + 0.0575866522821714*y1**4*y2**2 - 92.4845567577443*y1**4*y2*λ**4 - 60.1570291314254*y1**4*y2*λ**2 - 2.25964605360396e-9*y1**4*λ**3 - 6.65238197861129e-10*y1**4*λ - 187.786618803877*y1**3*y2**4*λ**4 - 174.602690093693*y1**3*y2**4*λ**2 - 0.690105237895676*y1**3*y2**4 - 1339.48713337051*y1**3*y2**3*λ**3 - 115.662265665606*y1**3*y2**3*λ - 985.438937667053*y1**3*y2**2*λ**4 - 767.560667055545*y1**3*y2**2*λ**2 - 0.451266217813104*y1**3*y2**2 - 428.161143538836*y1**3*y2*λ**3 - 21.5695881319482*y1**3*y2*λ - 0.234423134460347*y1**3*y2 - 3.62481722415708e-9*y1**3*λ**4 - 6.62604715540027e-9*y1**3*λ**2 - 0.0673400670172847*y1**3 - 329.597882060321*y1**2*y2**4*λ**3 - 30.186054951743*y1**2*y2**4*λ - 809.290948491726*y1**2*y2**3*λ**4 - 682.

In [29]:
profit_2 = (y2-w2)*(1-π**2)
profit_2

(1 - Min(1, Max(0, (-93.8706007741935*y1**4*y2**4*λ**3 - 9.17803097766009*y1**4*y2**4*λ - 226.19265377082*y1**4*y2**3*λ**4 - 207.454297053934*y1**4*y2**3*λ**2 - 0.843018528872131*y1**4*y2**3 - 471.265378957849*y1**4*y2**2*λ**3 - 37.3588280788617*y1**4*y2**2*λ + 0.0575866522821714*y1**4*y2**2 - 92.4845567577443*y1**4*y2*λ**4 - 60.1570291314254*y1**4*y2*λ**2 - 2.25964605360396e-9*y1**4*λ**3 - 6.65238197861129e-10*y1**4*λ - 187.786618803877*y1**3*y2**4*λ**4 - 174.602690093693*y1**3*y2**4*λ**2 - 0.690105237895676*y1**3*y2**4 - 1339.48713337051*y1**3*y2**3*λ**3 - 115.662265665606*y1**3*y2**3*λ - 985.438937667053*y1**3*y2**2*λ**4 - 767.560667055545*y1**3*y2**2*λ**2 - 0.451266217813104*y1**3*y2**2 - 428.161143538836*y1**3*y2*λ**3 - 21.5695881319482*y1**3*y2*λ - 0.234423134460347*y1**3*y2 - 3.62481722415708e-9*y1**3*λ**4 - 6.62604715540027e-9*y1**3*λ**2 - 0.0673400670172847*y1**3 - 329.597882060321*y1**2*y2**4*λ**3 - 30.186054951743*y1**2*y2**4*λ - 809.290948491726*y1**2*y2**3*λ**4 - 682.45583

Those were massive but that's the end of our notebook. As you can see, the price of precision is readability.